## Delta Live Tables - Gold Layer

In [0]:
expec_coaches = {
        "rule1" : "code is not null",
        "rule 2" : "current is true"
        }


In [0]:
expec_nocs = {
        "rule1" : "code is not null"
        }


In [0]:
expec_events = {
        "rule1" : "event is not null"
        }


In [0]:
import dlt
from pyspark.sql.functions import *

In [0]:
@dlt.table

def source_coaches():
    df = spark.readStream.table("olympicscatalog.silver.coaches")
    return df

In [0]:
@dlt.view

def view_coaches():
    df = spark.readStream.table("LIVE source_coaches")
    df = df.fillna("Unknown")
    return df

In [0]:
@dlt.table
@dlt.expect_all(expec_coaches)
def coaches():
    df = spark.readStream.table("LIVE view_coaches")
    return df

In [0]:
@dlt.view

def source_nocs():
    df = spark.readStream.table("olympicscatalog.silver.nocs")
    return df

In [0]:
@dlt.table
@dlt.expect_all_or_drop(expec_nocs)
def nocs():
    df = spark.readStream.table("LIVE source_nocs")
    return df

In [0]:
@dlt.view

def source_events():
    df = spark.readStream.table("olympicscatalog.silver.events")
    return df

In [0]:
@dlt.table
@dlt.expect_all(expec_events)

def events():
    df = spark.readStream.table("LIVE source_events")
    return df

In [0]:
@dlt.view

def source_athletes():
    df = spark.readStream.table("olympicscatalog.silver.athletes")
    return df 

In [0]:
dlt.create_streaming_table("athletes")

In [0]:
dlt.apply_changes(
    target = "athletes",
    source = "source_athletes",
    keys = "athlete_id",
    sequence_by = col("height"),
    stored_as_scd_type = 1
)